In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu115.html

In [3]:
!pip install git+https://github.com/SberProcessMining/Sber_Process_Mining.git
!pip install pm4py

  Created wheel for pyvis: filename=pyvis-0.2.1-py3-none-any.whl size=23667 sha256=b434f66441294b1d6b2f7e297d13ef6cb68462e3602aebed2d0636a0c243a094
  Stored in directory: /root/.cache/pip/wheels/2a/8f/04/6340d46afc74f59cc857a594ca1a2a14a1f4cbd4fd6c2e9306
  Created wheel for stringdist: filename=StringDist-1.0.9-cp37-cp37m-linux_x86_64.whl size=23603 sha256=62f6ddb8388a55ac68290e177b07e1519bf0a3f50883b9e7b9e3ea768b008976
  Stored in directory: /root/.cache/pip/wheels/d7/9c/d4/63bc3d8931de0980b9e4a724dea290bb40b8b1b2bd6227c8da
Successfully built pyvis stringdist


https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html

In [1]:
import os.path as osp
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import numpy as np
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import sys
sys.path.append('/content/drive/MyDrive/Drill planner/Code')
from Drill_planner import DrillPlaner, nx_graph, GraphMiner, DrillGraphMiner
import networkx as nx
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv, GAE
from torch_geometric.utils import train_test_split_edges

In [2]:
import torch_geometric as pyg

# Preparing graph dataset

In [3]:
data = pd.read_excel('/content/drive/MyDrive/Drill planner/Data/Dataset2.xlsx')

In [4]:
data['Operation code'] = data['Operation code'].astype(int)
burek = data[data['Phase'] == 'БУРЭК']
graph_planner = DrillGraphMiner(data, 'Time, h (in grains)', 'Operation code')
DFG =graph_planner.to_dfg()


In [5]:
print("Sparsity, %: ", 100 -np.round(np.count_nonzero(graph_planner.graph_matrix)/(graph_planner.graph_matrix.size),2)*100)

Sparsity, %:  95.0


In [6]:
edge_index = list(DFG.edges)


# Extract LCC

In [7]:
edge_index = pyg.utils.from_networkx(DFG).edge_index
edge_index

tensor([[  0,   1,   1,  ..., 141, 141, 141],
        [ 49,   6,  49,  ..., 121, 122, 134]])

In [8]:
data = pyg.data.Data(edge_index=edge_index)
nx_data = pyg.utils.to_networkx(data, to_undirected=True)
nx_data

/usr/local/lib/python3.7/dist-packages/torch_geometric/data/storage.py:266: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  " to suppress this warning")


In [9]:
connected_components = list(nx.connected_components(nx_data))
len(connected_components)
print(f"Graph contains {len(connected_components)} connected components")

Graph contains 2 connected components


In [10]:
lcc_nodes = max(connected_components, key=len)
lcc = nx_data.subgraph(lcc_nodes)
print(f"{lcc.number_of_nodes():,} nodes in LCC")

142 nodes in LCC


# Link prediction

In [11]:
indices = torch.arange(edge_index.shape[1])
train_idx, val_idx = train_test_split(indices, test_size=0.01)
train_mask, val_mask = torch.zeros_like(indices).bool(), torch.zeros_like(indices).bool()
train_mask[train_idx] = 1
val_mask[val_idx] = 1
train_mask, val_mask

(tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  

In [12]:
num_nodes = lcc.number_of_nodes()
x = torch.sparse.FloatTensor(
    indices=torch.stack([torch.arange(num_nodes), torch.arange(num_nodes)]),
    values=torch.ones(num_nodes),
)

data = pyg.data.Data(edge_index=edge_index, x=x, train_mask=train_mask, val_mask=val_mask)
data

Data(x=[142, 142], edge_index=[2, 960], train_mask=[960], val_mask=[960])

In [13]:
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[142, 142], edge_index=[2, 960])

In [ ]:
dataset = train_test_split_edges(data)

# Prepare model

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [58]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [59]:
out_channels = 2
num_features = 10
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.edge_index[:, data.train_mask].to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [36]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [63]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

RuntimeError: ignored

# Graph Edit distance

In [ ]:
nx.all_pairs_node_connectivity(DFG)
undire = nx.to_undirected(DFG)


preds = nx.jaccard_coefficient(undire, [('startevent', 302), (425, 202)])
for u, v, p in preds:
    print(f"({u}, {v}) -> {p:.8f}")

In [131]:
data['Operation code'] = data['Operation code'].astype(int)
burek = data[data['Phase'] == 'БУРЭК']
burkon = data[data['Phase'] == 'БУРКОН']
burpot = data[data['Phase'] == 'БУРПОТ']
burhv = data[data['Phase'] == 'БУРХВОСТ']
graph_planner1 = DrillGraphMiner(burek, 'Time, h (in grains)', 'Operation code')
DFG_burek =graph_planner.to_dfg()
graph_planner2 = DrillGraphMiner(burkon, 'Time, h (in grains)', 'Operation code')
DFG_burkon =graph_planner2.to_dfg()
graph_planner3 = DrillGraphMiner(burpot, 'Time, h (in grains)', 'Operation code')
DFG_burpot =graph_planner3.to_dfg()

graph_planner4 = DrillGraphMiner(burhv, 'Time, h (in grains)', 'Operation code')
DFG_burhv =graph_planner4.to_dfg()

In [133]:
nx.graph_edit_distance(nx.to_undirected(DFG_burek),nx.to_undirected(DFG_burkon))

KeyboardInterrupt: ignored

In [ ]:
nx.graph_edit_distance(DFG_burek,DFG_burpot)

In [ ]:
nx.graph_edit_distance(DFG_burek,DFG_burhv)